In [1]:
import numpy as np
import pandas as pd
import sys,os
import pylab
from scipy import stats,signal,io
import matplotlib.pyplot as plt
from sklearn import linear_model
import pickle
import seaborn as sns
sns.set_style('white')
sns.set_style('ticks')
import h5py
import matplotlib.gridspec as gridspec
plt.rcParams['pdf.fonttype'] = 'truetype'
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from functools import reduce
import datetime

import spikeinterface as si
import spikeinterface.extractors as se
import spikeinterface.toolkit as st
import spikeinterface.sorters as ss
import spikeinterface.comparison as sc
import spikeinterface.widgets as sw



In [2]:
base_path = '/n/groups/datta/guitchounts/data/gmou03/2020-07-30_11-02-59/'

In [3]:
data = se.OpenEphysRecordingExtractor(base_path)

recording = data.load_probe_file(probe_file='%s/../../32_groups.prb' % base_path)

Loading Open-Ephys: reading settings.xml...
Decoding data from  binary  format
.dat:  continuous.dat
Estimated samples:  108259200  Numchan:  32


In [4]:
print('Channel ids:', recording.get_channel_ids())
print('Loaded properties', recording.get_shared_channel_property_names())
print('Label of channel 0:', recording.get_channel_property(channel_id=0, property_name='label'))


recording_f = st.preprocessing.bandpass_filter(recording, freq_min=300, freq_max=6000)
recording_cmr = st.preprocessing.common_reference(recording_f, reference='median')


num_workers = 20


Channel ids: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
Loaded properties ['group', 'label', 'location']
Label of channel 0: t_0


In [5]:
default_ms4_params = ss.Mountainsort4Sorter.default_params()
#print(default_ms4_params)
ms4_params = default_ms4_params.copy()
ms4_params['adjacency_radius'] = 0
ms4_params['detect_sign'] = 0
ms4_params['filter'] = False
ms4_params['num_workers'] = num_workers
fs = 3e4




default_kl_params = ss.KlustaSorter.default_params()
#print(default_kl_params)
kl_params = default_kl_params.copy()
kl_params['adjacency_radius'] = 0
kl_params['extract_s_before'] = 32
kl_params['detect_sign'] = 0
#kl_params['filter'] = False
#kl_params['num_workers'] = num_workers



In [6]:
import time

In [ ]:
print('Starting mountainsort4 sorting....')

# run KlustaKwik:

start = time.time()



# run Mountainsort:
sorting_MS4 = ss.run_sorter('mountainsort4',recording_cmr, #grouping_property='group',
                       parallel=True,
                       output_folder='%s/tmp_MS4' % base_path, **ms4_params)

print('sorting finished in ',time.time() - start )


sorting_MS4 = st.curation.threshold_firing_rates(sorting_MS4, 
    duration_in_frames=recording_cmr.get_num_frames(), threshold=0.05, threshold_sign='less')

sorting_MS4 = st.curation.threshold_snrs(sorting_MS4, recording_cmr, threshold=5, threshold_sign='less')

st.postprocessing.export_to_phy(recording_cmr, sorting_MS4, output_folder='%s/phy_MS4' % base_path, grouping_property='group')



end = time.time()
print(end - start)

### started at 2:16 pm

Starting mountainsort4 sorting....
'end_time' set to 108259200
